# Ценообразование опционов. Часть 4.

# Экзотические опционы. Барьерные опционы.

## Цели и задачи:
### 1. Изучить подходы к моделированию барьерных опционов 
### 2. Получить навык работы с классами  Python
### 3. Получить дополнительные навыки работы с библиотеками NumPy и Pandas

### Рекомендуемая литература:
### 1.Levy G. Computational Finance. Numerical Methods for Pricing Financial Instruments
### 2.Neftci N.S. An Introduction to the Mathematics of Financial Derivatives
### 3.Hilpisch Y. Derivatives Analytics with Python: Data Analysis, Models, Simulation, Calibration and Hedging
### 4.London J. Modelling Derivatives with C++
### 5.Haug E.G. The Complete Guide to Option Pricing Formulas

## 1.Загрузка библиотек

In [1]:
import numpy as np
import matplotlib as plt
import math
from tqdm import tqdm # progress bar
from scipy import stats
import itertools
from scipy.linalg import cholesky

## 2.Классы, необходимые для реализации моделей ценообразования

### Геометрическое Броуновское движение (шаг)

In [2]:
class GBM_step:
    def __init__(self,S,drift,sigma,dt,div=0):
        self._S=S
        self._drift=drift
        self._sigma=sigma
        self._dt=dt
        self._div=div
        
    def __random_normal__(mu=0,sigma=1):
        return np.random.normal(0,1)
            
    def get_step(self):
            # S_current*=np.exp((drift-0.5*sigma**2)*dt+sigma*math.sqrt(dt)*dW)   
        return self._S*((self._drift-self._div)*self._dt+np.sqrt(self._dt)*self._sigma*self.__random_normal__())

### Monte Carlo барьер

In [3]:
class Barrier_MCS:
    def __init__(self,S,K,H,r,days,dt,sigma,rebate=0,div=0,is_call=True,is_up=True,is_in=True,is_barrier_american=True,nsim=10000):
        self._S=S
        self._H=H
        self._K=K
        self._r=r
        self._div=div
        self._days=days
        self._sigma=sigma
        self._dt=dt
        self._is_call=is_call 
        self._is_up=is_up
        self._is_in=is_in
        self._is_barrier_american=is_barrier_american
        self._rebate=rebate
        self._nsim=nsim     
        
    def __generate_payoff__(self):
        path=[]
        S_current=self._S
        path.append(S_current)
        for _ in range(self._days):  
            step=GBM_step(S_current,self._r,self._sigma,self._dt,self._div)
            S_current+=step.get_step() 
            path.append(S_current)     
        if self._is_barrier_american:
            S_min=min(path)
            S_max=max(path)
        else:S_min=S_max=path[-1]    
        if (~self._is_in&(((S_max<=self._H)&self._is_up)|((S_min>=self._H)&~self._is_up)))|\
        (self._is_in&(((S_max>=self._H)&self._is_up)|((S_min<=self._H)&~self._is_up))):
            return max(path[-1]-self._K,0) if self._is_call else max(self._K-path[-1],0)
        else: 
            return self._rebate
    
    def __simulation__(self):
        payoffs=[]
        for _ in tqdm(range(self._nsim)):
            payoffs.append(self.__generate_payoff__())
        return payoffs 
    
    def get_price(self):
        payoffs=[]
        payoffs=self.__simulation__()
        return np.mean(payoffs)*np.exp(-self._r*days/250)

### Барьер, модификация  Black-Scholes

In [4]:
class Barrier:
    def __init__(self,S,K,H,r,T,sigma,rebate=0,div=0,t=0,is_call=True,is_up=True,is_in=True):
        self._S=S
        self._H=H
        self._K=K
        self._r=r
        self._div=div
        self._T=T
        self._sigma=sigma
        self._t=t
        self._is_call=is_call 
        self._is_up=is_up
        self._is_in=is_in
        self._rebate=rebate
        self._eta=-1 if is_up else 1
        self._phi=1 if is_call else -1
        
    def __N__(self,value):
            return stats.norm.cdf(value)
    
    def __mu__(self):
        return (self._div-self._sigma**2/2)/self._sigma**2
    
    def __zet__(self,Lambda):
        return np.log(self._H/self._S)/(self._sigma*np.sqrt(self._T-self._t))+Lambda*self._sigma*np.sqrt(self._T-self._t)
     
    def __Lambda__(self,mu):
        return np.sqrt(mu**2+2*self._r/self._sigma**2)                             
    
    def __value_x_y__(self,mu):
        x_1=np.log(self._S/self._K)/(self._sigma*np.sqrt(self._T-self._t))+(1+mu)*self._sigma*np.sqrt(self._T-self._t)
        x_2=np.log(self._S/self._H)/(self._sigma*np.sqrt(self._T-self._t))+(1+mu)*self._sigma*np.sqrt(self._T-self._t)
        y_1=np.log(self._H**2/(self._K*self._S))/(self._sigma*np.sqrt(self._T-self._t))+(1+mu)*self._sigma*np.sqrt(self._T-self._t)
        y_2=np.log(self._H/self._S)/(self._sigma*np.sqrt(self._T-self._t))+(1+mu)*self._sigma*np.sqrt(self._T-self._t)     
        return x_1,x_2,y_1,y_2
    
    def __values__(self):
        print(self._eta,self._phi)
        mu=self.__mu__()
        Lambda=self.__Lambda__(mu)
        z=self.__zet__(Lambda)
        x_1,x_2,y_1,y_2=self.__value_x_y__(mu)
        
        a=self._phi*self._S*np.exp((self._div-self._r)*(self._T-self._t))
        b=self._phi*self._K*np.exp(-self._r*(self._T-self._t))
        c=self._phi*x_1-self._phi*self._sigma*np.sqrt(self._T-self._t)
        d=self._phi*x_2-self._phi*self._sigma*np.sqrt(self._T-self._t)
        e=(self._H/self._S)**(2*(mu+1))
        f=(self._H/self._S)**(2*mu)
        g=self._eta*y_1-self._eta*self._sigma*np.sqrt(self._T-self._t)
        h=self._eta*y_2-self._eta*self._sigma*np.sqrt(self._T-self._t)
        i=self._eta*x_2-self._eta*self._sigma*np.sqrt(self._T-self._t)
        j=self._eta*z-2*self._eta*Lambda*self._sigma*np.sqrt(self._T-self._t)
        k=(self._H/self._S)**(mu+Lambda)
        l=(self._H/self._S)**(mu-Lambda)
        
        A=a*self.__N__(self._phi*x_1)-b*self.__N__(c)
        
        B=a*self.__N__(self._phi*x_2)-b*self.__N__(d)
        
        C=a*e*self.__N__(self._eta*y_1)-b*f*self.__N__(g)
        
        D=a*e*self.__N__(self._eta*y_2)-b*f*self.__N__(h)
        
        E=(self._rebate*np.exp(-self._r*(self._T-self._t)))*(self.__N__(i)-f*self.__N__(h))
        
        F=self._rebate*(k*self.__N__(self._eta*z)+l*self.__N__(j))
        
        return A,B,C,D,E,F
    
    def __price__(self):
        A,B,C,D,E,F=self.__values__()
        price=0
        if self._is_in:
            if self._is_call:  
                if self._is_up: # up-and-in call
                    if self._K>self._H:
                        price=A+E
                    else: price=B-C+D+E    
                else:    # down-and-in call
                    if self._K>self._H:
                        price=C+E
                    else: price=A-B+D+E
            else:
                if self._is_up: # up-and-in put
                    if self._K>self._H:
                        price=A-B+D+E
                    else: price=C+E
                else:    # down-and-in put
                    if self._K>self._H:
                        price=B-C+D+E
                    else: price=A+E
        else:
            if self._is_call:
                if self._is_up: # up-and-out call
                    if self._K>self._H:
                        price=F
                    else: price=A-B+C-D+F 
                else:# down-and-out call
                    if self._K>self._H:
                        price=A-C+F
                    else: price=B-D+F
            else:
                if self._is_up: # up-and-out put
                    if self._K>self._H:
                        price=B-D+F
                    else: price=A-C+F  
                else:# down-and-out put
                    if self._K>self._H:
                        price=A-B+C-D+F
                    else: price=F
        return price
      
    
    def get_price(self):
        return self.__price__()

## Реализация методов

In [5]:
S_0=100
K=110
H=100
sigma=0.3
T=125/250
r=0.08
div=0.04
rebate=3
days=125
dt=1/250

In [6]:
is_call=True
is_up=False
is_in=True

### Модификация Black Scholes

In [7]:
call_do=Barrier(S_0,K,H,r,T,sigma,rebate=rebate,div=div,is_call=is_call,is_up=is_up,is_in=is_in)

In [8]:
call_do.get_price()

1 1


5.3043012601780575

### Монте-Карло

In [9]:
call_do=Barrier_MCS(S_0,K,H,r,days,dt,sigma,rebate=rebate,div=div,is_call=is_call,is_up=is_up,is_in=is_in,is_barrier_american=True,\
                    nsim=10000)

In [10]:
call_do.get_price()

100%|██████████| 10000/10000 [00:06<00:00, 1565.84it/s]


5.464860691405411